# Legal document classification in zero-shot cross lingual transfer setting

# Part II: Results reproduction

Date: May 2025

Project of course: Natural Language Processing - ENSAE 3A S2

Author: Noémie Guibé

In [ ]:
# imports
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import pandas as pd 
from datasets import Dataset
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score
import os

from src import baseline_model, frozen_model, adapter_model

2025-05-03 03:53:43.964908: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-03 03:53:43.965764: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-03 03:53:43.970454: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-03 03:53:43.982717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746244424.003592  151792 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746244424.01

In [2]:
# Taining parameters
train_size = 5000
test_size = 5000
batch_size = 32
epochs = 2

In [3]:
# import data base
df = pd.read_parquet('https://minio.lab.sspcloud.fr/nguibe/NLP/multi_eurlex_reduced.parquet', engine='pyarrow')

# 1 - First result reproduction: Performance drop from English-only fine-tuning

In [4]:
# Taining parameters
train_size = 5
test_size = 5
batch_size = 32
epochs = 1

In [8]:
# Run training and evaluation
results = baseline_model.run_training_pipeline(data=df,train_sample_size=train_size,
                                test_sample_size=test_size,
                                batch_size=batch_size,
                                epochs=epochs)

# Display results
import pprint
pprint.pprint(results)

Map: 100%|██████████| 1/1 [00:00<00:00, 28.53 examples/s]
All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: in user code:

    File "/usr/local/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1381, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.12/site-packages/transformers/modeling_tf_utils.py", line 1652, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.12/site-packages/tf_keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_fileztupeatj.py", line 40, in tf__run_call_with_unpacked_inputs
        raise
    File "/tmp/__autograph_generated

## Performance analysis

Investigating catastrophic forgetting more deeply by comparing the weight shifts between a pre-trained multilingual model and a retrained, English-only model could provide a more granular understanding of how language-specific information is lost during retraining. 

In [ ]:
def compute_weight_difference(pretrained_weights, retrained_weights):
    differences = []
    for pretrained, retrained in zip(pretrained_weights, retrained_weights):
        diff = np.linalg.norm(pretrained - retrained)  # L2 norm for weight difference
        differences.append(diff)
    return differences


In [ ]:
# Retrieve weights
pretrained_weights = model.get_weights()  # Pretrained model weights
retrained_weights = model.get_weights()   # Retrained model weights

# Compute the differences between pre-trained and retrained model
weight_differences = compute_weight_difference(pretrained_weights, retrained_weights)


In [ ]:
# Visualize or identify which layers show the most significant changes
most_changed_layers = np.argsort(weight_differences)[::-1]


3.	Visualize the Results:

o	You could create a heatmap or bar chart to visualize the magnitude of the changes in each layer.

o	Layer-wise comparison (e.g., attention heads, feed-forward layers, etc.) could reveal which part of the model "forgot" multilingual information the most.

4.	Interpretation of Results:

o	By examining the most significantly changed layers, you could hypothesize which parts of the model are most sensitive to language-specific data. For example:

	Attention layers might "forget" multilingual information, especially if they were tuned heavily toward English sentence structure.

	Embedding layers might change more dramatically as they encode language-specific token representations.

o	Comparing only certain parts of the network, like the early transformer layers (closer to embeddings), versus the deeper layers (closer to output), could give insights into how the model's ability to generalize across languages is disrupted.

________________________________________
Challenges:

•	Scale: Since models like XLM-Roberta are large (hundreds of millions of parameters), computing and storing weight differences across all layers could be resource-intensive.

•	Layer-specific analysis: Some layers (e.g., final layers or task-specific heads) may change more due to the retraining task (e.g., classification), so you’d want to focus on layers like attention and embeddings that are more involved in the multilingual representation.

•	Learning rate and retraining settings: Differences in fine-tuning settings (like learning rate) could also cause changes in specific parts of the model


# 2 - Second result reproduction: "better" performance with adaptation strategies

## Frozen layers

In [7]:
N= 6

In [8]:
# Run training and evaluation of model with N frozen layers and same other parameters
results = frozen_model.run_training_pipeline_with_freezing(df=df,train_sample_size=train_size,
                                test_sample_size=test_size,
                                batch_size=batch_size,
                                epochs=epochs, n_frozen_layer= N)

# Display results
import pprint
pprint.pprint(results)

Map: 100%|██████████| 1/1 [00:00<00:00, 28.52 examples/s]
All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[INFO] Successfully froze first 6 transformer layers.
1/1 [==============================] - 21s 21s/step - loss: 0.6903 - auc_1: 0.1667


2025-05-03 03:56:40.414919: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


{'final_memory': 13611.21875,
 'initial_memory': 9474.7578125,
 'model_params': 278059797,
 'results': {'de': {'LRAP': 0.39490411549235077,
                    'Macro F1': 0.09523809523809523,
                    'Micro F1': 0.4444444444444444,
                    'R-Precision': 0.2},
             'en': {'LRAP': 0.5909722222222222,
                    'Macro F1': 0.07936507936507936,
                    'Micro F1': 0.4615384615384615,
                    'R-Precision': 0.30000000000000004},
             'fi': {'LRAP': 0.23452380952380952,
                    'Macro F1': 0.047619047619047616,
                    'Micro F1': 0.28571428571428575,
                    'R-Precision': 0.2},
             'fr': {'LRAP': 0.1568890056022409,
                    'Macro F1': 0.0,
                    'Micro F1': 0.0,
                    'R-Precision': 0.0}},
 'training_time': 20.674821138381958}


## Adaptaters

In [12]:
# Taining parameters
train_size = 5000
test_size = 5000
batch_size = 32
epochs = 2

In [ ]:
# Run training and evaluation of model with N frozen layers and same other parameters
results = adapter_model.run_adapter_training_pipeline(data=df,train_sample_size=train_size,
                                test_sample_size=test_size,
                                batch_size=batch_size,
                                epochs=epochs)

# Display results
import pprint
pprint.pprint(results)

Map:   0%|          | 0/1003 [00:00<?, ? examples/s]